# Profiling and discussion

## Lorenz 96

#### First Version: Understanding Data Types and Profiling
- **Initial Approach**: Started with a basic implementation, focusing on getting the logic correct.
- **Key Learning**: Realized the importance of specifying data types for optimization in NumPy.
- **Performance**: Profiling with `timeit` showed a runtime of 1.03 milliseconds for a certain array size.

#### Second Version: Embracing Vectorization
- **Optimization Attempt**: Replaced nested loops with NumPy's vectorization capabilities.
- **Trade-off**: Noticed slower performance for smaller step sizes but gains in larger simulations.
- **Performance**: Runtime improved to 228 microseconds for larger steps.

#### Third Version: Memory Efficiency
- **Consultation Insights**: ChatGPT suggested improvements in memory management.
- **Change Implemented**: Avoided unnecessary array creation for each iteration.
- **Result**: The performance did not improve as expected, indicating a need for further refinement.

#### Fourth Version: Strategic Vectorization
- **Algorithmic Adjustment**: Divided the computation into sections to optimize the vectorization process.
- **Outcome**: Achieved significant speed improvements without compromising the model's integrity.
- **Performance**: Solidified the gains, maintaining a runtime of 228 microseconds.

### Post-Assignment Enhancements

#### Continued Learning and Application
- **Advanced Techniques**: After the assignment, I delved into more advanced NumPy features to improve performance further.
- **Code Quality**: Refactored the code for better readability and maintainability, aligning with Python's best practices.

#### Algorithmic Refinements
- **Boundary Conditions**: Investigated more efficient methods for handling periodic boundary conditions.
- **Pseudocode/Diagrams**: [Include pseudocode or diagrams of any new algorithmic approaches here.]

#### Emphasizing Code Quality
- **Refactoring**: Modularized the code for better clarity and reuse.
- **Commenting**: Enhanced comments and documentation for future users and contributors.

During the assigment I was not able to specify which array sizes I specifically tested for but now I will test the function at the time of assigment with my improved one post assignment for a small array and a larger one.

Old version of function:

In [1]:
# Add profiling code here
import numpy as np


def lorenz96_old(initial_state, nsteps, constants=(1 / 101, 100, 8)):
    """
    Perform iterations of the Lorenz 96 update.

    Parameters
    ----------
    initial_state : array_like or list
        Initial state of lattice in an array of floats.
    nsteps : int
        Number of steps of Lorenz 96 to perform.

    Returns
    -------
    numpy.ndarray
        Final state of lattice in an array of floats
    """

    alpha, beta, gamma = constants
    state = np.array(initial_state, dtype=float)
    N = len(state)
    new_state = np.empty_like(state)  # Create a new state array

    for _ in range(nsteps):
        new_state[0] = alpha * (
            (beta * state[0]) + (state[N - 2] - state[1]) * state[N - 1] + gamma
        )

        # Compute the second element
        new_state[1] = alpha * (
            beta * state[1] + (state[0] - state[2]) * state[N - 1] + gamma
        )
        
        # Compute the elements between 2 and N-2
        new_state[2:N - 1] = alpha * (
            beta * state[2:N - 1] +
            (state[0:N - 3] - state[3:N]) * state[1:N - 2] +
            gamma
        )

        # Compute the last element
        new_state[N - 1] = alpha * (
            beta * state[N - 1] + (state[N - 3] - state[0]) * state[N - 2] + gamma
        )

        # Update the state array
        state[:] = new_state


    return state

In [2]:
initial_state = np.full(49, 8.0) # Create an array of 49 8.0s
initial_state = np.insert(initial_state, 2, 9.0) # Insert a 9.0 at index 2
nsteps = 50 # Number of steps to perform

In [3]:
from automata import lorenz96
%timeit lorenz96(initial_state, nsteps)

726 µs ± 52.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [4]:
%timeit lorenz96_old(initial_state, nsteps)

233 µs ± 3.38 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [5]:
initial_state = np.full(9, 8.0) # Create an array of 49 8.0s
initial_state = np.insert(initial_state, 2, 9.0) # Insert a 9.0 at index 2
nsteps = 1 # Number of steps to perform

In [6]:
print(lorenz96(initial_state, nsteps))
print()
print(lorenz96_old(initial_state, nsteps))

[7.92079208 8.         8.99009901 8.07920792 8.         8.
 8.         8.         8.         8.        ]

[8.         7.92079208 8.99009901 8.         8.07920792 8.
 8.         8.         8.         8.        ]


In [7]:
# Add profiling code here
import cProfile


pr = cProfile.Profile()
pr.enable()
lorenz96(initial_state, nsteps)
pr.disable()

pr.print_stats(sort='time')

         105 function calls (93 primitive calls) in 0.000 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      6/3    0.000    0.000    0.000    0.000 numeric.py:1147(roll)
        3    0.000    0.000    0.000    0.000 {built-in method numpy.array}
        1    0.000    0.000    0.000    0.000 automata.py:9(lorenz96)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
      9/3    0.000    0.000    0.000    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        3    0.000    0.000    0.000    0.000 numeric.py:1348(normalize_axis_tuple)
      6/3    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(roll)
        2    0.000    0.000    0.000    0.000 interactiveshell.py:3490(run_code)
        2    0.000    0.000    0.000    0.000 traitlets.py:689(__get__)
        2    0.000    0.000    0.000    0.000 codeop.py:117(__call__)
        2    0.000    0.

This was a test with a small array so the effects of vectorisation is not apparent. Lets try with a bigger array:

In [8]:
# Example of larger initial state and more steps
initial_state = np.full(999, 8.0)  # A larger array
initial_state = np.insert(initial_state, 2, 9.0)  # Insert a 9.0 at index 2
nsteps = 50  # More steps

In [9]:
%timeit lorenz96(initial_state, nsteps)

817 µs ± 2.18 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [10]:
%timeit lorenz96_old(initial_state, nsteps)

318 µs ± 3.41 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Game of Life

### Initial Implementation Challenges:
The original Conway's Game of Life code provided by ChatGPT faced numerous issues, including bugs and missing logic for specific rules like the "2d colour" extension. The initial version lacked efficiency and robustness, necessitating a thorough revision and enhancement process.

### Structural Refinement and Validation:
To improve reliability and functionality, a robust skeleton was developed. This included validating array inputs and correctly handling different game dimensions and rules. The primary focus was refining the 2D version of the game, setting a strong foundation for further extensions and optimizations.

### Strategic Consolidation:
While the initial guidance from ChatGPT suggested separate functions for different rules and dimensions, a unified approach was adopted. This consolidation aimed to reduce the overhead from multiple function calls, focusing on optimizing execution time as measured by the `timeit` function. While this approach initially decreased readability, it was essential for performance gains.

### Advanced Optimizations (After assignment):
After establishing a working codebase, several advanced optimization techniques were applied to enhance performance further:

- **Simplifying Conditionals:** The code was refactored to streamline conditionals, reducing computational overhead and increasing clarity.
- **Efficient Boundary Management:** Periodic boundary conditions were handled more effectively using modular arithmetic, cutting down on unnecessary checks and conditions.
- **Leveraging NumPy:** The adoption of NumPy for array operations significantly boosted performance, replacing slower Python loops with efficient, vectorized operations.

### Modular and Readable Code:
Post-assignment enhancements focused on not just performance but also maintainability and readability. The code was refactored into modular functions, each handling a specific aspect of the game logic. This made the code easier to understand, maintain, and extend, without sacrificing the performance gains achieved earlier.

### Algorithmic Refinements and Testing:
Further algorithmic refinements were made to handle edge cases and optimize for various scenarios. The code was extensively tested to ensure accuracy and performance across a wide range of conditions and parameters. Special attention was given to array sizes, boundary conditions, and different rule sets to validate the implementation comprehensively.


In [11]:
import numpy as np

def life_old(initial_state, nsteps, rules="basic", periodic=False):
    """
    Perform iterations of Conway’s Game of Life.
    Parameters
    ----------
    initial_state : array_like or list of lists
        Initial 2d state of grid in an array of ints.
    nsteps : int
        Number of steps of Life to perform.
    rules : str
        Choose a set of rules from "basic", "2colour" or "3d".
    periodic : bool
        If True, use periodic boundary conditions.
    Returns
    -------
    numpy.ndarray
         Final state of grid in array of ints.
    """

    # write your code here to replace return statement
    state = np.array(initial_state, dtype=int)

    # Determine if we're working in 2D or 3D
    if rules == "3d":
        if state.ndim != 3:
            raise ValueError("Invalid grid dimension!")
        rows, cols, depth = state.shape
        for _ in range(nsteps):
            next_state = state.copy()

            for i in range(rows):
                for j in range(cols):
                    for k in range(depth):
                        total = 0  # Count the neighbors

                        for x in [-1, 0, 1]:
                            for y in [-1, 0, 1]:
                                for z in [-1, 0, 1]:
                                    if x == 0 and y == 0 and z == 0:
                                        continue  # Skip the current cell
                                    ni, nj, nk = i + x, j + y, k + z
                                    if periodic:  # Handle periodic boundary conditions
                                        ni %= rows
                                        nj %= cols
                                        nk %= depth
                                    elif (
                                        ni < 0
                                        or ni >= rows
                                        or nj < 0
                                        or nj >= cols
                                        or nk < 0
                                        or nk >= depth
                                    ):
                                        continue  # Skip out of bounds

                                    total += state[ni, nj, nk]

                            if state[i, j, k] != 0:  # If cell is alive
                                if total < 5 or total > 6:  # Die
                                    next_state[i, j, k] = 0
                            else:  # Dead cell
                                if total == 4:
                                    next_state[i, j, k] = 1  # Birth

            state = next_state
        return state
    else:
        if state.ndim != 2:
            raise ValueError("Invalid grid dimension!")
        rows, cols = state.shape
        for _ in range(nsteps):
            next_state = state.copy()

            for i in range(rows):
                for j in range(cols):
                    total = 0
                    blue_neighbours = 0
                    red_neighbours = 0
                    for x in [-1, 0, 1]:
                        for y in [-1, 0, 1]:
                            if x == 0 and y == 0:
                                continue
                            ni, nj = i + x, j + y
                            if periodic:
                                ni %= rows
                                nj %= cols
                            elif ni < 0 or ni >= rows or nj < 0 or nj >= cols:
                                continue

                            if state[ni][nj] > 0:
                                total += 1
                                if state[ni][nj] == 1:
                                    blue_neighbours += 1
                                else:
                                    red_neighbours += 1

                    if rules == "basic":
                        if state[i][j] == 1:  # If cell is alive
                            if total < 2 or total > 3:  # Die
                                next_state[i][j] = 0
                        else:  # Dead cell
                            if total == 3:
                                next_state[i][j] = 1

                    elif rules == "2colour":
                        if state[i][j] == 1 or state[i][j] == 2:  # If cell is alive
                            if total < 2 or total > 3:  # Die
                                next_state[i][j] = 0
                        else:  # Dead cell
                            if total == 3:
                                if blue_neighbours > red_neighbours:
                                    next_state[i][j] = 1
                                else:
                                    next_state[i][j] = 2

            state = next_state

        return state


initial_state = (
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1],
)

In [12]:
from automata import life
%timeit life(initial_state, 10000, rules="basic", periodic=False)

276 ms ± 7.26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
initial_state = np.array([[0, 0, 0, 0], [0, 1, 0, 0], [0, 1, 2, 0], [0, 1, 0, 0]])

In [14]:
# Sanity check to see if the old and new functions give the same result
print(life(initial_state, 1, rules="2colour", periodic=False))
print()
print(life_old(initial_state, 1, rules="2colour", periodic=False))

[[0 0 0 0]
 [0 1 0 0]
 [1 1 2 0]
 [0 1 0 0]]

[[0 0 0 0]
 [0 1 1 0]
 [1 1 2 0]
 [0 1 1 0]]
